<a href="https://colab.research.google.com/github/dsynderg/479_Final_Project/blob/main/ModelUser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install OpenNMT-py
!pip install "numpy<2.0" # This fixes an error caused by OpenNMT not being maintained
import random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.8/262.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.8/37.8 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 124.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
modelname = "modle_step_180000.pt"

In [2]:
outputname = "2000_check"

In [3]:
language = "complete"

In [4]:
translate_yaml_text = f"""
model: /content/{modelname}
src: /content/src_test.txt
output: /content/{outputname}
gpu: 0
verbose: True

# Sentencepiece Params

# Tokenization options
transforms: onmt_tokenize
src_subword_type: sentencepiece
src_subword_model: english_sp_model.model
tgt_subword_type: sentencepiece
tgt_subword_model: {language}_sp_model.model

src_subword_nbest: 20
src_subword_alpha: 0.1
"""

with open("translate_yaml.yaml", "w") as f:
    f.write(translate_yaml_text)

In [5]:
!onmt_translate -config translate_yaml.yaml

Streaming output truncated to the last 5000 lines.
PRED 1001: ▁I <unk> , ▁esaki ▁ta ▁un ▁importante ▁relato ▁di <unk> ▁tokante ▁e ▁uso ▁di ▁e ▁uso ▁di ▁e <unk> nan ▁di ▁e <unk> nan .
PRED SCORE: -0.8322

[2025-12-10 17:22:21,123 INFO] 
SENT 1002: ['▁<', 'English', '>', '▁É', 's', 'tos', '▁pueden', '▁abarca', 'r', '▁la', '▁popular', 'idad', '▁', 'mund', 'ana', ',', '▁el', '<unk>', '▁público', ',', '▁las', '▁pro', 'e', 'za', 's', '▁física', 's', ',', '▁el', '▁talento', '▁art', 'ístico', '▁o', '▁de', 'por', 'tivo', ',', '▁la', '▁prosperidad', '▁y', '▁las', '▁riquezas', '.']
PRED 1002: ▁These ▁may ▁include ▁popular ity , ▁the ▁public <unk> , ▁the ▁physical <unk> , ▁the ▁artist ic <unk> , ▁or <unk> , ▁prosper ity , ▁and ▁riches .
PRED SCORE: -0.5550

[2025-12-10 17:22:21,124 INFO] 
SENT 1003: ['▁<', 'Portuguese', '>', '▁What', '▁', 'i', 's', '▁Heavenly', '▁Father', "'", 's', '▁message', '▁for', '▁me', '▁in', '▁this', '▁verse', '?']
PRED 1003: ▁O ▁que ▁a ▁mensagem ▁do ▁Pai ▁Celestial ▁para ▁

In [6]:
!python -m pip install sacrebleu

In [7]:
from sacrebleu import corpus_bleu

with open("/content/tgt_test.txt", encoding="utf-8") as f:
    refs = [r.strip() for r in f]

with open(outputname, encoding="utf-8") as f:
    hyps = [h.strip() for h in f]

assert len(hyps) == len(refs), "Files must have same number of lines"

score = corpus_bleu(hyps, [refs])  # references must be a list-of-lists
print(f"BLEU = {score.score:.2f}, details: {score.format()}")

BLEU = 36.25, details: BLEU = 36.25 60.5/41.2/30.3/22.9 (BP = 1.000 ratio = 1.113 hyp_len = 45671 ref_len = 41019)


In [ ]:
!python -m pip install sacrebleu

In [8]:
# example: compute_chrf.py
from sacrebleu import corpus_chrf

with open("tgt_test.txt", encoding="utf-8") as f:
    refs = [line.strip() for line in f if line.strip()]

with open(outputname, encoding="utf-8") as f:
    hyps = [line.strip() for line in f if line.strip()]

assert len(hyps) == len(refs), "Files must have same number of lines"

# corpus_chrf expects hyps, [refs]; beta controls precision/recall tradeoff (default 2.0)
score = corpus_chrf(hyps, [refs], beta=2.0)
print(f"chrF = {score.score:.4f}")

chrF = 59.7886
